This notebook takes the StatsBomb json files and turns them into parquet files. These are extremely fast to load so good for this prototyping kind of analysis.

# This update
This analysis has been re-run from the commit 2381d6b0afbf59b93c0a41dd4cc017f4cbbc4d9b:
https://github.com/statsbomb/open-data/commit/2381d6b0afbf59b93c0a41dd4cc017f4cbbc4d9b.

From this commit, the dataframes have the following number of entries:

- df_competition: 42 entries
- df_match: 1,242 entries (10 event files aren't included in the match file)
- df_lineup: 42,035 entries
- df_event: 4,420,124 entries
- df_freeze: 400,976 entries
- df_tactic: 53,933 entries
- df_related: 8,521,814 entries


# Rerunning the analysis from the original thesis:
Note the original analysis for this thesis was run from the commit 87a5f02d7f526c4fe92909790999da5f26166328:
https://github.com/statsbomb/open-data/commit/87a5f02d7f526c4fe92909790999da5f26166328. If you want to re-run the thesis analysis checkout the commit with the command: git checkout 87a5f02d7f526c4fe92909790999da5f2616632.

In [19]:
from mplsoccer import Sblocal
import pandas as pd
import os
import glob
import numpy as np
from tqdm import tqdm

# Change these paths/ parameters
You will need to change these paths/ parameters depending on where the StatsBomb open-data is located, how and where you want to save the resulting data, and if you only want the new files to be processed.

In [20]:
# open data folder is one folder down in the directory. To change if run elsewhere
STATSBOMB_DATA = os.path.join('..', '..', '..', 'open-data', 'data')
# save files in folder in current directory. To change if want to save elsewhere
DATA_FOLDER = os.path.join('..', '..', 'data', 'statsbomb')

# Get the data file paths

In [21]:
event_links = glob.glob(os.path.join(STATSBOMB_DATA, 'events', '**', '*.json'), recursive=True)
lineup_links = glob.glob(os.path.join(STATSBOMB_DATA, 'lineups', '**', '*.json'), recursive=True)
match_links = glob.glob(os.path.join(STATSBOMB_DATA, 'matches', '**', '*.json'), recursive=True)
competition_path = os.path.join(STATSBOMB_DATA, 'competitions.json')

# Make the directory structure

In [22]:
# make the directory structure
for folder in ['event_raw', 'related_raw', 'freeze_raw', 'tactic_raw', 'lineup_raw']:
    path = os.path.join(DATA_FOLDER, folder)
    if not os.path.exists(path):
        os.mkdir(path)

# Create StatsBomb Parser

In [23]:
parser = Sblocal()

# Read the competition data

In [24]:
df_competition = parser.competition(competition_path)
df_competition.to_parquet(os.path.join(DATA_FOLDER, 'competition.parquet'), allow_truncated_timestamps=True)
df_competition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   competition_id             74 non-null     int64 
 1   season_id                  74 non-null     int64 
 2   country_name               74 non-null     object
 3   competition_name           74 non-null     object
 4   competition_gender         74 non-null     object
 5   competition_youth          74 non-null     bool  
 6   competition_international  74 non-null     bool  
 7   season_name                74 non-null     object
 8   match_updated              74 non-null     object
 9   match_updated_360          56 non-null     object
 10  match_available_360        10 non-null     object
 11  match_available            74 non-null     object
dtypes: bool(2), int64(2), object(8)
memory usage: 6.1+ KB


# Keep a copy of the match dataframe

In [25]:
match_path = os.path.join(DATA_FOLDER, 'match.parquet')
if os.path.exists(match_path):
    df_match_copy = pd.read_parquet(match_path).copy()
    UPDATE_FILES = True
else:
    UPDATE_FILES = False

# Read the match data

In [26]:
match_dfs = [parser.match(file) for file in match_links]
df_match = pd.concat(match_dfs)
# again there is a slight loss of quality when saving timestamps, but only relevant for last_updated
df_match.to_parquet(os.path.join(DATA_FOLDER, 'match.parquet'),
                    allow_truncated_timestamps=True)
df_match.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3433 entries, 0 to 33
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   match_id                         3433 non-null   int64         
 1   match_date                       3433 non-null   datetime64[ns]
 2   kick_off                         3428 non-null   datetime64[ns]
 3   home_score                       3433 non-null   int64         
 4   away_score                       3433 non-null   int64         
 5   match_status                     3433 non-null   object        
 6   match_status_360                 3433 non-null   object        
 7   last_updated                     3433 non-null   datetime64[ns]
 8   last_updated_360                 1797 non-null   datetime64[ns]
 9   match_week                       3433 non-null   int64         
 10  competition_id                   3433 non-null   int64         
 11

# Check which games have been updated

In [27]:
if UPDATE_FILES:
    df_match_copy = (df_match[['match_id', 'last_updated']]
                     .merge(df_match_copy[['match_id', 'last_updated']],
                            how='left', suffixes=['', '_old'], on='match_id'))
    df_match_copy = df_match_copy[(df_match_copy.last_updated.dt.floor('ms') !=
                                   df_match_copy.last_updated_old.dt.floor('ms'))].copy()
    to_update = df_match_copy.match_id.unique()

    # get array of event links to update - based on whether they have been updated in the match json
    event_link_ids = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]
    event_to_update = [link in to_update for link in event_link_ids]
    event_links = np.array(event_links)[event_to_update]

    # get array of lineup links to update -
    # based on whether they have been updated in the match jsons
    lineup_link_ids = [int(os.path.splitext(os.path.basename(link))[0]) for link in lineup_links]
    lineup_to_update = [link in to_update for link in lineup_link_ids]
    lineup_links = np.array(lineup_links)[lineup_to_update]

# Read the lineup data

In [28]:
LINEUP_FOLDER = os.path.join(DATA_FOLDER, 'lineup_raw')
# loop through all the changed links and store as parquet files - small and fast files
for file in tqdm(lineup_links):
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        df_lineup = parser.lineup(file)
        df_lineup.to_parquet(os.path.join(LINEUP_FOLDER, save_path))
    except ValueError:
        print('Skipping file:', file)

100%|██████████| 3433/3433 [01:11<00:00, 48.23it/s]


Convert to a single dataframe

In [29]:
# players with split ids
to_replace = {18103: 38522}  # Dietmar Hamann

In [30]:
if len(lineup_links) == 0:
    print('No update')
else:
    lineup_files = glob.glob(os.path.join(LINEUP_FOLDER, '*.parquet'))
    df_lineup = pd.concat([pd.read_parquet(file) for file in tqdm(lineup_files)])
    # replace some ids that appear to be split
    df_lineup.player_id.replace(to_replace, inplace=True)
    df_lineup.to_parquet(os.path.join(DATA_FOLDER, 'lineup.parquet'))
    df_lineup.info()

100%|██████████| 3433/3433 [00:53<00:00, 64.45it/s] 


<class 'pandas.core.frame.DataFrame'>
Index: 130488 entries, 0 to 35
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   player_id        130488 non-null  int64  
 1   player_name      130488 non-null  object 
 2   player_nickname  130488 non-null  object 
 3   jersey_number    130488 non-null  int64  
 4   match_id         130488 non-null  int64  
 5   team_id          130488 non-null  int64  
 6   team_name        130488 non-null  object 
 7   country_id       130475 non-null  float64
 8   country_name     130475 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 10.0+ MB


C:\Users\nicol\AppData\Local\Temp\ipykernel_29060\1261813975.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_lineup.player_id.replace(to_replace, inplace=True)


# Read the event data

In [31]:
# loop through all the changed links and store as parquet files - small and fast files
for file in tqdm(event_links):
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        events, related, freeze, tactics = parser.event(file)
        # save to parquet files
        # using the dictionary key to access the dataframes from the dictionary
        events.to_parquet(os.path.join(DATA_FOLDER, 'event_raw', save_path))
        related.to_parquet(os.path.join(DATA_FOLDER, 'related_raw', save_path))
        freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze_raw', save_path))
        tactics.to_parquet(os.path.join(DATA_FOLDER, 'tactic_raw', save_path))
    except ValueError:
        print('Skipping:', file)

100%|██████████| 3433/3433 [19:28<00:00,  2.94it/s]


If updating the event dataframe get a list of ids to update

In [32]:
event_files = glob.glob(os.path.join(DATA_FOLDER, 'event_raw', '*.parquet'))
if UPDATE_FILES:
    ids_to_update = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]

Function to load old dataframe and combine with updated parquet files

In [33]:
def update(directory, file_type, update_ids):
    """ Update an old DataFrame with files that have changed/ been added.
    Parameters
    ----------
    directory : path to directory containing the files
    file_type : str
        One of 'event', 'freeze', 'tatic', or related'
    update_ids : list of integers
        A list of the match ids to update
    Returns
    -------
    df : pandas.DataFrame
        An updated DataFrame with the new/changed matches.
    """
    # get a list of parquet files to add to the old dataframe
    files = glob.glob(os.path.join(directory, f'{file_type}_raw', '*.parquet'))
    files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in files]
    mask_update = [match_id in update_ids for match_id in files_id]
    files = np.array(files)[mask_update]
    # load the old dataframe, filter out changed matches and add the new parquet files
    df_old = pd.read_parquet(os.path.join(directory, f'{file_type}.parquet'))
    df_old = df_old[~df_old.match_id.isin(update_ids)]
    df_new = pd.concat([pd.read_parquet(file) for file in tqdm(files)])
    df_old = pd.concat([df_old, df_new])
    return df_old

Single dataframe events

In [34]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_event = update(DATA_FOLDER, 'event', ids_to_update)
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, show_counts=True)
    else:
        df_event = pd.concat([pd.read_parquet(file) for file in tqdm(event_files)])
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, show_counts=True)

100%|██████████| 3433/3433 [01:37<00:00, 35.09it/s]
C:\Users\nicol\AppData\Local\Temp\ipykernel_29060\3132692631.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split


<class 'pandas.core.frame.DataFrame'>
Index: 12083338 entries, 0 to 3656
Data columns (total 90 columns):
 #   Column                          Non-Null Count     Dtype  
---  ------                          --------------     -----  
 0   id                              12083338 non-null  object 
 1   index                           12083338 non-null  int64  
 2   period                          12083338 non-null  int64  
 3   timestamp                       12083338 non-null  object 
 4   minute                          12083338 non-null  int64  
 5   second                          12083338 non-null  int64  
 6   possession                      12083338 non-null  int64  
 7   duration                        8907451 non-null   float64
 8   match_id                        12083338 non-null  int64  
 9   type_id                         12083338 non-null  int64  
 10  type_name                       12083338 non-null  object 
 11  possession_team_id              12083338 non-null  int64 

Single dataframe shot freeze frames

In [35]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_freeze = update(DATA_FOLDER, 'freeze', ids_to_update)
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info(verbose=True, show_counts=True)
    else:
        freeze_files = glob.glob(os.path.join(DATA_FOLDER, 'freeze_raw', '*.parquet'))
        df_freeze = pd.concat([pd.read_parquet(file) for file in tqdm(freeze_files)])
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info()

100%|██████████| 3433/3433 [00:58<00:00, 58.98it/s]
C:\Users\nicol\AppData\Local\Temp\ipykernel_29060\294442557.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split


<class 'pandas.core.frame.DataFrame'>
Index: 1110839 entries, 0 to 341
Data columns (total 10 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   teammate         1110839 non-null  bool   
 1   match_id         1110839 non-null  int64  
 2   id               1110839 non-null  object 
 3   x                1110839 non-null  float64
 4   y                1110839 non-null  float64
 5   player_id        1110839 non-null  int64  
 6   player_name      1110839 non-null  object 
 7   position_id      1110839 non-null  int64  
 8   position_name    1110839 non-null  object 
 9   event_freeze_id  1110839 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 85.8+ MB


Single dataframe tactics

In [36]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_tactic = update(DATA_FOLDER, 'tactic', ids_to_update)
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info(verbose=True, show_counts=True)
    else:
        tactic_files = glob.glob(os.path.join(DATA_FOLDER, 'tactic_raw', '*.parquet'))
        df_tactic = pd.concat([pd.read_parquet(file) for file in tqdm(tactic_files)])
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info()

100%|██████████| 3433/3433 [00:50<00:00, 68.39it/s]


<class 'pandas.core.frame.DataFrame'>
Index: 169598 entries, 0 to 54
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   jersey_number     169598 non-null  int64 
 1   match_id          169598 non-null  int64 
 2   id                169598 non-null  object
 3   player_id         169598 non-null  int64 
 4   player_name       169598 non-null  object
 5   position_id       169598 non-null  int64 
 6   position_name     169598 non-null  object
 7   event_tactics_id  169598 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 11.6+ MB


C:\Users\nicol\AppData\Local\Temp\ipykernel_29060\1203663716.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split


Single dataframe related events

In [37]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_related = update(DATA_FOLDER, 'related', ids_to_update)
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, show_counts=True)
    else:
        related_files = glob.glob(os.path.join(DATA_FOLDER, 'related_raw', '*.parquet'))
        df_related = pd.concat([pd.read_parquet(file) for file in tqdm(related_files)])
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, show_counts=True)

100%|██████████| 3433/3433 [01:13<00:00, 46.55it/s]


<class 'pandas.core.frame.DataFrame'>
Index: 22969422 entries, 0 to 5232
Data columns (total 7 columns):
 #   Column             Non-Null Count     Dtype 
---  ------             --------------     ----- 
 0   match_id           22969422 non-null  int64 
 1   id                 22969422 non-null  object
 2   index              22969422 non-null  int64 
 3   type_name          22969422 non-null  object
 4   id_related         22969422 non-null  object
 5   index_related      22969422 non-null  int64 
 6   type_name_related  22969422 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.4+ GB
